In [1]:
%pip install pandas scikit-learn numpy google-cloud-bigquery hopsworks hsfs[python] optuna xgboost

  Using cached confluent_kafka-1.9.0-cp39-cp39-win_amd64.whl (3.0 MB)
  Attempting uninstall: confluent-kafka
    Found existing installation: confluent-kafka 2.1.1
    Uninstalling confluent-kafka-2.1.1:
      Successfully uninstalled confluent-kafka-2.1.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\diego\\anaconda3\\envs\\curso\\Lib\\site-packages\\~onfluent_kafka.libs\\libcrypto-3-x64-1f376d5f9507bc3330a5f961bb54c621.dll'
Consider using the `--user` option or check the permissions.



In [13]:
import pandas as pd
demography_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/demography.csv")
customer_info_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/customer_info.csv")
subscriptions_df = pd.read_csv("https://repo.hops.works/dev/davit/churn/subscriptions.csv")

In [14]:
def convert_dt(df):   
    df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
    df['clienteidentifier'] = df['clienteidentifier'].astype(str)
    df['multiplelines'] = df['multiplelines'].astype(str)
    df['internetservice'] = df['internetservice'].astype(str)
    df['onlinesecurity'] = df['onlinesecurity'].astype(str)
    df['onlinebackup'] = df['onlinebackup'].astype(str)
    df['deviceprotection'] = df['deviceprotection'].astype(str)
    df['techsupport'] = df['techsupport'].astype(str)
    df['streamingtv'] = df['streamingtv'].astype(str)
    df['streamingmovies'] = df['streamingmovies'].astype(str)
    df['contract'] = df['contract'].astype(str)
    df['paymentmethod'] = df['paymentmethod'].astype(str)
    df['gender'] = df['gender'].astype(str)
    df['paperlessbilling'] = df['paperlessbilling'].astype(str)
    df['partner'] = df['partner'].astype(str)
    df['dependents'] = df['dependents'].astype(str)
    df['phoneservice'] = df['phoneservice'].astype(str)
    df['seniorcitizen'] = df['seniorcitizen'].astype(str)
    df['monthlycharges'] = df['monthlycharges'].astype(float)
    df['totalcharges'] = df['totalcharges'].astype(float)
    df['tenure'] = df['tenure'].astype(float)
    df['churn'] = df['churn'].astype(str)
    return df
    #df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso']).dt.date
    
def reindex_df(df):
    new_order = ['clienteidentifier','multiplelines','internetservice','onlinesecurity','onlinebackup',
             'deviceprotection','techsupport','streamingtv','streamingmovies','contract','paymentmethod',
             'gender','paperlessbilling','partner','dependents','phoneservice','seniorcitizen',
             'monthlycharges','totalcharges','tenure','churn']
    
    data = df.reindex(columns=new_order)
    return data

In [15]:
# Realizar el join por el campo ID
df = customer_info_df.merge(subscriptions_df, on='customerID').merge(demography_df, on='customerID')
df = pd.DataFrame(df)
df.columns = df.columns.str.lower()
df = df.rename(columns={'customerid': 'clienteidentifier'})
# Imprimir el resultado


In [16]:
df = convert_dt(df)
data = reindex_df(df)

In [17]:
data

,clienteidentifier,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,...,gender,paperlessbilling,partner,dependents,phoneservice,seniorcitizen,monthlycharges,totalcharges,tenure,churn
0,7590-VHVEG,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,...,Female,Yes,Yes,No,No,0,29.85,29.85,1.0,No
1,5575-GNVDE,No,DSL,Yes,No,Yes,No,No,No,One year,...,Male,No,No,No,Yes,0,56.95,1889.50,34.0,No
2,3668-QPYBK,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,...,Male,Yes,No,No,Yes,0,53.85,108.15,2.0,Yes
3,7795-CFOCW,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,...,Male,No,No,No,No,0,42.30,1840.75,45.0,No
4,9237-HQITU,No,Fiber optic,No,No,No,No,No,No,Month-to-month,...,Female,Yes,No,No,Yes,0,70.70,151.65,2.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,...,Male,Yes,Yes,Yes,Yes,0,84.80,1990.50,24.0,No
7039,2234-XADUH,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,...,Female,Yes,Yes,Yes,Yes,0,103.20,7362.90,72.0,No
7040,4801-JZAZL,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,...,Female,Yes,Yes,Yes,No,0,29.60,346.45,11.0,No
7041,8361-LTMKD,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,...,Male,Yes,Yes,No,Yes,1,74.40,306.60,4.0,Yes


In [20]:
## CREAR FEATURE GROUP PRIMERO
churn_input = fs.get_or_create_feature_group(name = "churn_input_train",
                                            version = 1,
                                            primary_key = ["clienteidentifier"],
                                            description = "Training feature group"
                                            )
churn_input.insert(data)

Uploading Dataframe: 0.00% |          | Rows 0/7043 | Elapsed Time: 00:00 | Remaining Time: ?

KafkaException: KafkaError{code=_TRANSPORT,val=-195,str="Failed to get metadata: Local: Broker transport failure"}

## Training Pipeline

In [21]:
## TRANSFORMADOR AUXILIAR
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

def impute_missing_values(data):
    # Variables categóricas
    categorical_cols = data.select_dtypes(include='object').columns
    #categorical_cols = categorical_cols.drop('churn')  # Excluir 'churn'
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    data[categorical_cols] = categorical_imputer.fit_transform(data[categorical_cols])

    # Variables numéricas
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    numeric_imputer = SimpleImputer(strategy='mean')
    data[numeric_cols] = numeric_imputer.fit_transform(data[numeric_cols])

    return data

def normalize_numeric_variables(data):
    numeric_cols = ['monthlycharges', 'totalcharges', 'tenure']

    scaler = MinMaxScaler()
    data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

    return data

def map_categorical_variables(data):
    mapping = {
        'multiplelines': {'No': 0, 'Yes': 0.5, 'No phone service': 1},
        'internetservice': {'Fiber optic': 0, 'DSL': 0.5, 'No': 1},
        'onlinesecurity': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'onlinebackup': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'deviceprotection': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'techsupport': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'streamingtv': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'streamingmovies': {'No': 0, 'Yes': 0.5, 'No internet service': 1},
        'contract': {'Month-to-month': 0, 'One year': 0.5, 'Two year': 1},
        'gender' : {'Male' : 0, 'Female': 1},
        'paymentmethod': {'Electronic check': 0, 'Mailed check': 0.3, 'Bank transfer (automatic)': 0.6, 'Credit card (automatic)': 1},
        'paperlessbilling' : {'Yes' : 1, 'No': 0},
        'partner' : {'Yes' : 1, 'No': 0},
        'dependents' : {'Yes' : 1, 'No': 0},
        'phoneservice' : {'Yes' : 1, 'No': 0},
        'seniorcitizen' : {'0' : 0, '1': 1},
        'churn' : {'Yes' : 1, 'No': 0},    
    }

    data.replace(mapping, inplace=True)

    return data

def remove_duplicates(data):
    data.drop_duplicates(inplace=True)
    return data

In [7]:
data = impute_missing_values(data)
data = remove_duplicates(data)
data =  normalize_numeric_variables(data)
data = map_categorical_variables(data)
data = data.drop('clienteidentifier', axis=1)

In [8]:
data

,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paymentmethod,gender,paperlessbilling,partner,dependents,phoneservice,seniorcitizen,monthlycharges,totalcharges,tenure,churn
0,1.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,0,0,0,0.115423,0.001275,0.013889,0
1,0.0,0.5,0.5,0.0,0.5,0.0,0.0,0.0,0.5,0.3,0,0,0,0,1,0,0.385075,0.215867,0.472222,0
2,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.3,0,1,0,0,1,0,0.354229,0.010310,0.027778,1
3,1.0,0.5,0.5,0.0,0.5,0.5,0.0,0.0,0.5,0.6,0,0,0,0,0,0,0.239303,0.210241,0.625000,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,1,0,0.521891,0.015330,0.027778,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.5,0.3,0,1,1,1,1,0,0.662189,0.227521,0.333333,0
7039,0.5,0.0,0.0,0.5,0.5,0.0,0.5,0.5,0.5,1.0,1,1,1,1,1,0,0.845274,0.847461,1.000000,0
7040,1.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,0,0,0.112935,0.037809,0.152778,0
7041,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0,1,1,0,1,1,0.558706,0.033210,0.055556,1


  Using cached confluent_kafka-2.1.1-cp39-cp39-win_amd64.whl (3.4 MB)
  Attempting uninstall: confluent-kafka
    Found existing installation: confluent-kafka 1.9.0
    Uninstalling confluent-kafka-1.9.0:
      Successfully uninstalled confluent-kafka-1.9.0


In [19]:
!pip install --upgrade hsfs confluent_kafka

In [12]:
import hopsworks 
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/61594
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
try:
    feature_view = fs.get_feature_view(name = "churn_input_train", version = 1)
except:
    churn_inp = fd.get_feature_group(name = "churn_input_train", version = 1)
    query = churn_inp.select_all()
    feature_view = fs.create_feature_view(name = "churn_input_train",
                                         version = 1,
                                         description = "Leer de churn",
                                         labels = ["churn"],
                                         query = query)

## Optimización de Hiperparametros

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import xgboost as xgb
from xgboost import XGBoostClassifier
import optuna

# Función objetivo para la optimización de hiperparámetros con Optuna
def objective(trial):
    params = {
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 3),
        'max_depth' : trial.suggest_int('max_depth', 3, 10),
        'n_estimators' : trial.suggest_int('n_estimators', 100, 1000, step=100),
    }

    # Código para cargar tus datos
    # X contiene las características de entrada y y contiene las etiquetas objetivo
    X = data.drop('churn', axis = 1)
    y = data['churn']

    # Lista para almacenar los resultados de evaluación
    f1_scores = []
    accuracies = []
    confusion_matrices = []

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Utilizamos 5-fold cross-validation

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)

        # Realizar predicciones en conjunto de prueba
        y_pred = model.predict(X_test)

        # Calcular métricas de evaluación
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        confusion = confusion_matrix(y_test, y_pred)

        # Almacenar los resultados de evaluación
        f1_scores.append(f1)
        accuracies.append(accuracy)
        confusion_matrices.append(confusion)

    return np.mean(f1_scores)  # Puedes ajustar aquí la métrica que deseas optimizar

# Ejecutar la optimización de hiperparámetros con Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # Especifica el número de iteraciones que desees

# Imprimir los mejores hiperparámetros encontrados por Optuna
print("Mejores hiperparámetros:")
print(study.best_params)

In [ ]:
# Imprimir los mejores hiperparámetros encontrados por Optuna
print("Mejores hiperparámetros:")
print(study.best_params)

## Entrenamiento final del model

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

#dividir conjunto de datos
X = data.drop('churn', axis = 1)
y = data['churn']

# instancia parametros y modelo
params = study.best_params
xgb_model = xgb.XGBClassifier(**params)

# instancial metricas de evaluación
f1_scores = []
accuracies = []
confusion_matrices = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    xgb_model.fit(X_train, y_train)

    # Realizar predicciones en conjunto de prueba
    y_pred = xgb_model.predict(X_test)

    # Calcular métricas de evaluación
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)

    # Almacenar los resultados de evaluación
    f1_scores.append(f1)
    accuracies.append(accuracy)
    confusion_matrices.append(confusion)



In [ ]:
# Imprimir los resultados
print("Resultados de evaluación en todo el conjunto de datos:")
print("F1 score:", f1)
print("Accuracy:", accuracy)
print("Confusion matrix:", confusion)

In [ ]:
xgb_model

In [ ]:
import os
import pickle

# Ruta del directorio actual (ajustar según la ubicación del script en el repositorio)
dir_path = os.path.join(os.getcwd(), "src")

# Ruta de la carpeta "models"
models_dir = os.path.join(dir_path, "..", "models")

# Crear la carpeta "models" si no existe
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Ruta del archivo de modelo dentro de la carpeta "models"
model_path = os.path.join(models_dir, "xgb_predictor.pkl")

# Guardar el modelo en disco
with open(model_path, 'wb') as file:
    pickle.dump(xgb_model, file)



In [ ]:
# Cargar el modelo desde el archivo
with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
!pip install --upgrade hsfs confluent_kafka

In [ ]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()


In [ ]:
query = """
SELECT *
FROM `protean-fabric-386717.ml_datasets.churn_last`
WHERE fecha_ingreso = CURRENT_DATE()
"""

In [ ]:
df = fs.get_feature_group("churn_input")

In [ ]:
df.select_all().show(10)

In [ ]:
#guardar modelo en carperta
import os
import pickle

# Ruta del directorio actual (donde se encuentra el script.py)
dir_path = os.path.dirname(os.path.abspath(__file__))

# Ruta de la carpeta "models"
models_dir = os.path.join(dir_path, "..", "models")

# Crear la carpeta "models" si no existe
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Ruta del archivo de modelo dentro de la carpeta "models"
model_path = os.path.join(models_dir, "xgb_predictor.pkl")

# Guardar el modelo en disco
with open(model_path, 'wb') as file:
    pickle.dump(xgb_model, file)

# Cargar el modelo desde el archivo
with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)